# Process / Todo:

* [x] Convert results to GPKG and reproject to web mercator
* -> [x] Hardcode source and target CRS
* [ ] Generate patch squares as polygons
* -> [x] Hardcode patch dimensions

* [x] Rules for overlapping patches
s
* [ ] Interpolate raster? 
* -> [ ] raster cell at approx half the patch dimensions?
* -> [ ] Generate tiles

* [ ] Disvole patches?
* -> Rules for overlapping patches
* [ ] Generate vector tiles


In [2]:
# %conda create --name geo_env --yes
# %conda activate geo_env
# %conda config --env --add channels conda-forge
# %conda config --env --set channel_priority strict
# %conda install python=3 geopandas --yes

# Install ogr2ogr if required
# /opt/homebrew/bin/brew install gdal
# %conda install -c conda-forge gdal
# %conda install -c conda-forge fiona
# %pip install centerline

In [3]:
import geopandas
from pathlib import Path
import pandas
from icecream import ic


# Globals / Parameters

In [4]:
mr_results_dir = Path(r"/Users/a.smith/Library/CloudStorage/OneDrive-SharedLibraries-TheAlanTuringInstitute/Living with Machines - team - Documents/MapReader/202111_files/results_v003")
display_data_dir = Path(r"/Users/a.smith/Documents.OLD/Living-with-machines/mapreader_display_data")

source_crs = "EPSG:4326" # WGS 1984
target_crs = "EPSG:3857" # Web mercator
patch_width = 200 # unit=>meters. This must be in the unit of the target_crs

# Convert results to GPKG and reproject to web mercator

In [5]:
# import glob
# import os

# # Convert everything to GPKG

# # Get all the file names and paths
# all_patches_csv = str(mr_results_dir / "all_patches_latlonpred.csv")
# # all_patches_geojson = str(display_data_dir / "all_patches.geojson" )
# all_patches_geopkg = str(display_data_dir / "all_patches2.gpkg" )
# pred_csv_list = glob.glob("pred_*.csv", dir_fd=mr_results_dir)
# pred_gpkg = str(display_data_dir / "pred2.gpkg" )

# # !ogr2ogr -f GeoJSON "$all_patches_geojson" "$all_patches_csv" -oo X_POSSIBLE_NAMES=center_lon -oo Y_POSSIBLE_NAMES=center_lat -oo KEEP_GEOM_COLUMNS=NO

# # Convert the main "all_patches.csv" 
# !ogr2ogr -f GPKG -t_srs "$target_crs" "$all_patches_geopkg" "$all_patches_csv" -s_srs "$source_crs" -oo X_POSSIBLE_NAMES=center_lon -oo Y_POSSIBLE_NAMES=center_lat -oo KEEP_GEOM_COLUMNS=YES

# # Now convert the "pred" (predared?) csv files, add each as a different layer
# for fname in pred_csv_list:
#     fpath = str(mr_results_dir / fname) 
#     lyr_name = os.path.splitext(fname)[0]
#     !ogr2ogr -overwrite -f GPKG -t_srs "$target_crs" "$pred_gpkg" -doo IDENTIFIER="$lyr_name" "$fpath" -s_srs "$source_crs" -oo X_POSSIBLE_NAMES=center_lon -oo Y_POSSIBLE_NAMES=center_lat -oo KEEP_GEOM_COLUMNS=YES


# # target CRS `-t_srs`
# # source CRS `-s_srs`

# # ic(file_list)
# # ic(csv_list)


# Rules for overlapping patches

In [6]:
# Rules for combining classifications from two (or more) patches, which overlap, due to overlapping map sheets.

"""
The function below defines rules for combining classifications from two (or more) patches, which overlap, due to overlapping map sheets.

This is a native implenmentation which:
* Treats each patch value with equal weight
* Always returns in favor of finding rail and or buildings.
If one patch is classified as containing rail, and the other patch is classified as containing nothing, then a rail space is returned.
If one patch is classified as containing rail, and the other patch is classified as containing building, then a rail AND building space is returned.

Future idea (not currently implenmented): A more sophificated approach might be to review the confidence index of each patch and select the value from the patch with the highest confidence.
"""

# For reference, the categories are:
# *   0 => No rail, nor buildings
# *   1 => Railspace, no buildings
# *   2 => Buildings without railspace
# *   3 => Railspace with buildings

# The funciton below relies on the two values in the tuple key being in assending order.
_patch_combinations = {
    (0, 0) : 0,
    (0, 1) : 1,
    (0, 2) : 2,
    (0, 3) : 3,
    (1, 1) : 1,
    (1, 2) : 3, # Is this is wise assumption?
    (1, 3) : 3,
    (2, 2) : 2,
    (2, 3) : 3,
    (3, 3) : 3 
}

def get_overlapping_patch_value(pred_A, pred_B):
    # Sort the two pred values
    pred_pair = (pred_A, pred_B)
    if pred_A > pred_B:
        pred_pair = (pred_B, pred_A)

    return _patch_combinations[pred_pair]

# ic(get_overlapping_patch_value(0,0))
# ic(get_overlapping_patch_value(0,3))
# ic(get_overlapping_patch_value(1,2))
# ic(get_overlapping_patch_value(2,0))



# Generate patch squares as polygons

In [7]:
"""
# Adapted from here https://pcjericks.github.io/py-gdalogr-cookbook/vector_layers.html#save-centroids-of-input-layer-to-an-output-layer
"""
from osgeo import ogr
import os
import fiona
from shapely.geometry import box, mapping

# Input data (the points)
pred_gpkg = str(display_data_dir / "pred2.gpkg" )

ic(pred_gpkg)

# Output layer
pred_patches_ppkg = str(display_data_dir / "pred_squares2.gpkg" )

def patch_from_centroids(in_point, patch_width):
    # Half width of the patches (for convenience)
    patch_semi_width = 0.5 * patch_width
    out_feat = in_point
    cen_X, cen_Y = in_point['geometry']['coordinates']

    poly = box(
        cen_X - patch_semi_width,
        cen_Y - patch_semi_width,
        cen_X + patch_semi_width,
        cen_Y + patch_semi_width
    )
    out_feat['geometry'] = mapping(poly)
    return out_feat

def generate_patches_from_centroids(in_gpkg, in_lyr, out_gpkg, out_lyr):
    with fiona.open(pred_gpkg, layer=in_lyr) as point_lyr:
        dest_driver = point_lyr.driver
        dest_crs = point_lyr.crs
        dest_schema = point_lyr.schema

        # Change from point to polygon
        dest_schema["geometry"] = "Polygon"

        with fiona.open(
            out_gpkg,
            "w",
            layer=out_lyr,
            driver=dest_driver,
            crs=dest_crs,
            schema=dest_schema
        ) as polygon_lyr:
            i = 0
            out_feat_list = []
            for in_feat in point_lyr:
                i = i+1
                out_feat_list.append(patch_from_centre_point(in_feat, patch_width))

                # Write out features 10,000 at a time to balance memory vs write speed
                if i % 10000 == 0:
                    ic(f"generated {i} patches from centriods")
                    polygon_lyr.writerecords(out_feat_list)
                    out_feat_list = []

            # Write out any remaining features (assuming that hte number of features is not an exact multiple of 10000)
            ic(f"generated {i} patches from centriods")
            polygon_lyr.writerecords(out_feat_list)
            out_feat_list = []


# generate_patches_from_centroids(
#     in_gpkg=pred_gpkg,
#     in_lyr="pred_0103_all_v003",
#     out_gpkg=pred_patches_ppkg,
#     out_lyr="patches_0103_all_v003"
# )


# generate_patches_from_centroids(
#     in_gpkg=pred_gpkg,
#     in_lyr="pred_02_all_v003",
#     out_gpkg=pred_patches_ppkg,
#     out_lyr="patches_02_all_v003"
# )

ic| pred_gpkg: '/Users/a.smith/Documents.OLD/Living-with-machines/mapreader_display_data/pred2.gpkg'


In [8]:

def dummy():
    """
    # # Create the output Layer
    # outDriver = ogr.GetDriverByName("GPKG")

    # # Remove output shapefile if it already exists
    # if os.path.exists(pred_patches_ppkg):
    #     outDriver.DeleteDataSource(pred_patches_ppkg)

    # # Create the output shapefile
    # outDataSource = outDriver.CreateDataSource(pred_patches_ppkg)
    # outLayer = outDataSource.CreateLayer("states_centroids", geom_type=ogr.wkbPolygon)

    # # Add input Layer Fields to the output Layer
    # inLayerDefn = inLayer.GetLayerDefn()
    # ic(inLayerDefn)
    # for i in range(0, inLayerDefn.GetFieldCount()):
    #     fieldDefn = inLayerDefn.GetFieldDefn(i)
    #     outLayer.CreateField(fieldDefn)

    # # Get the output Layer's Feature Definition
    # outLayerDefn = outLayer.GetLayerDefn()

    # ic(inLayer.GetFeatureCount())
    # # inLayer.

    # # Add features to the ouput Layer
    # for i in range(2, inLayer.GetFeatureCount()):
    #     # Get the input Feature
    #     inFeature = inLayer.GetFeature(i)
    #     ic(inFeature)
    #     # Create output Feature
    #     outFeature = ogr.Feature(outLayerDefn)
    #     # Add field values from input Layer
    #     for i in range(0, outLayerDefn.GetFieldCount()):
    #         outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), inFeature.GetField(i))
    #     # Set geometry as centroid
    #     geom = inFeature.GetGeometryRef()
    #     inFeature = None
    #     centroid = geom.Centroid()
    #     ic(centroid)

    #     # Create ring
    #     ring = ogr.Geometry(ogr.wkbLinearRing)
    #     ring.AddPoint(cen_X - patch_semi_width, cen_Y - patch_semi_width)
    #     ring.AddPoint(cen_X - patch_semi_width, cen_Y + patch_semi_width)
    #     ring.AddPoint(cen_X + patch_semi_width, cen_Y + patch_semi_width)
    #     ring.AddPoint(cen_X + patch_semi_width, cen_Y - patch_semi_width)
    #     ring.AddPoint(cen_X - patch_semi_width, cen_Y - patch_semi_width)

    #     # Create polygon
    #     poly = ogr.Geometry(ogr.wkbPolygon)
    #     poly.AddGeometry(ring)

    #     outFeature.SetGeometry(poly)
    #     # Add new feature to output Layer
    #     outLayer.CreateFeature(outFeature)
    #     outFeature = None
    #     ring = None
    #     poly = None

    # # Save and close DataSources
    # inDataSource = None
    # outDataSource = None
    """
    pass

# Dissolve patches

In [9]:
# Switch to geopandas to disolve
import geopandas as gpd

dissolved_gpkg = str(display_data_dir / "dissolved_gpkg.gpkg" )

def dissolve_patches(in_gpkg, in_lyr, out_gpkg, out_lyr, group_by):
    patches_gdf = gpd.read_file(in_gpkg, layer=in_lyr)
    regions_gdf = patches_gdf.dissolve(by=group_by)
    regions_gdf.to_file(out_gpkg, layer=out_lyr, driver="GPKG")


# dissolve_patches(
#     in_gpkg=pred_patches_ppkg,
#     in_lyr="patches_0103_all_v003",
#     out_gpkg=dissolved_gpkg,
#     out_lyr="dissolved_0103",
#     group_by="pred"
# )

# dissolve_patches(
#     in_gpkg=pred_patches_ppkg,
#     in_lyr="patches_02_all_v003",
#     out_gpkg=dissolved_gpkg,
#     out_lyr="dissolved_02",
#     group_by="pred"
# )

# dissolve_patches(
#     in_gpkg=dissolved_gpkg,
#     in_lyr="dissolved_0103",
#     out_gpkg=dissolved_gpkg,
#     out_lyr="dissolve_all_rail",
#     group_by=None
# )


# Create centre lines

In [18]:
from shapely.geometry import Polygon, MultiPolygon, shape
from centerline.geometry import Centerline

# patches_gdf = gpd.read_file(in_gpkg, layer=in_lyr)

def _centerline_from_polygon(in_poly):
    # Half width of the patches (for convenience)
    out_feat = in_poly
    # ic(in_poly)
    # poly = Polygon(in_poly['geometry']['coordinates'])
    poly = shape(in_poly['geometry'])
    #  ic(in_poly['geometry'])
    ic(poly)
    centerline = Centerline(poly)
    out_feat['geometry'] = mapping(centerline)
    return out_feat

def centrelines_from_all_patches(in_gpkg, in_lyr, out_gpkg, out_lyr):
    with fiona.open(in_gpkg, layer=in_lyr) as polygon_lyr:
        dest_driver = polygon_lyr.driver
        dest_crs = polygon_lyr.crs
        dest_schema = polygon_lyr.schema

        # Change from polygon to 
        dest_schema["geometry"] = "MultiLineString"

        with fiona.open(
            out_gpkg,
            "w",
            layer=out_lyr,
            driver=dest_driver,
            crs=dest_crs,
            schema=dest_schema
        ) as cline_lyr:

            i = 0
            out_feat_list = []
            for in_feat in polygon_lyr:
                i = i+1
                out_feat_list.append(_centerline_from_polygon(in_feat))

                # Write out features 10,000 at a time to balance memory vs write speed
                if i % 10000 == 0:
                    ic(f"generated {i} patches from centriods")
                    cline_lyr.writerecords(out_feat_list)
                    out_feat_list = []

            # Write out any remaining features (assuming that hte number of features is not an exact multiple of 10000)
            ic(f"generated {i} patches from centriods")
            cline_lyr.writerecords(out_feat_list)
            out_feat_list = []


# centrelines_from_all_patches(
#     in_gpkg=dissolved_gpkg,
#     in_lyr="dissolve_all_rail",
#     out_gpkg=dissolved_gpkg,
#     out_lyr="rail_center_lines"
# )


test_data = str(display_data_dir / "pred_squares_extract.gpkg" )

centrelines_from_all_patches(
    in_gpkg=test_data,
    in_lyr="all_rail_extract",
    out_gpkg=test_data,
    out_lyr="rail_center_lines"
)



ic| in_poly: {'geometry': {'coordinates': [[[(-178252.6450411975, 7370528.188727902),
                                             (-178276.7734091605, 7370528.188727902),
                                             (-178428.51667323452, 7370528.188727902),
                                             (-178452.6450411975, 7370528.188727902),
                                             (-178604.38830527154, 7370528.188727902),
                                             (-178628.51667323452, 7370528.188727902),
                                             (-178780.25993730855, 7370528.188727902),
                                             (-178804.38830527154, 7370528.188727902),
                                             (-178956.1315693456, 7370528.188727902),
                                             (-178980.25993730855, 7370528.188727902),
                                             (-179132.0032013826, 7370528.188727902),
                                             (-1

# Generate Raster

In [ ]:
raise NotImplementedError("Not implemented below this cell")

# Vector tiles

In [ ]:
# Create the vector tiles
tile_names =  "all-patches.mbtiles"
all_patches_mbtiles = str(display_data / tile_names)
# Currently using the default options as givenn in https://github.com/mapbox/tippecanoe#readme
!/opt/homebrew/Cellar/tippecanoe/1.36.0/bin/tippecanoe -zg -o "$all_patches_mbtiles" --drop-densest-as-needed  "$all_patches_geojson"

In [ ]:
# Run a tile server with the vector tiles
tile_names =  "test-vector-tiles.mbtiles"
!docker run -d -it -v "$display_data":/data -p 8080:80 klokantech/tileserver-gl $tile_names